In [1]:
import os
from langchain_community.document_loaders.text import TextLoader
from langchain_community.document_loaders import DirectoryLoader
from langchain_experimental.text_splitter import SemanticChunker
from langchain_text_splitters import RecursiveCharacterTextSplitter
from yandex_chain import YandexEmbeddings
from langchain_community.vectorstores import FAISS

In [2]:
if os.getenv("environment") != "production":
    from dotenv import load_dotenv
    load_dotenv("../app/.env")

AUTH_DATA = os.getenv("AUTH_DATA")
API_KEY = os.getenv("API_KEY")
FOLDER_ID = os.getenv("FOLDER_ID")

In [3]:
loader = DirectoryLoader("../data/csv_structured",
                    glob="**/*.csv",
                    loader_cls=TextLoader,
                    show_progress=True,
                    loader_kwargs={"encoding":"utf-8"})

data = loader.load()

100%|██████████| 182/182 [00:00<00:00, 3710.74it/s]


In [4]:
text_splitter=RecursiveCharacterTextSplitter( # TODO: semantic chunking
    chunk_size=500, # TODO: tune this
    chunk_overlap=10
)

chunks = text_splitter.split_documents(data)

In [5]:
semantic_text_splitter = SemanticChunker(YandexEmbeddings(folder_id=FOLDER_ID, api_key=API_KEY))
chunks = text_splitter.split_documents(data)

In [6]:
faiss_vdb = "faiss_structured_v0.0.1"

In [7]:
db = FAISS.from_documents(chunks, YandexEmbeddings(folder_id=FOLDER_ID, api_key=API_KEY))
db.save_local(os.path.join("..", "data", "structured_vdb", faiss_vdb))

# inference

In [8]:
db = FAISS.load_local(
    os.path.join("..", "data", "structured_vdb", faiss_vdb),
    YandexEmbeddings(folder_id=FOLDER_ID, api_key=API_KEY),
    allow_dangerous_deserialization = True
)

In [13]:
query = "Какова дебиторская задолженность на 31 декабря 2022 г.?"
docs = db.similarity_search(query)

In [14]:
docs

[Document(page_content='Ha 31.12.2022,Ha 31.12.2021,\nДолгосрочная кредиторская задолженность,,\nОоязательства перед генеральным подрядчиком,269 478,447 222\nв том числе удёржано из оплаты генерального подрядчика,,\nГарантийное удержание,269 478,447 822\nПрочие удержания,,\nитого долгосрочная кредиторская задолженность:,1 269 478,447 822\nКраткосточнаа unaпuтоnе,,\nItHu кыuкнах креди торская задолженность V1 1 - -,,\nАванса получснныи по концессионному соглашению Обазяте 7т,909 191,', metadata={'source': '../data/csv_structured/scanned/27.04.2023 АЗ ГД МСФО за 2022г_опт/table_page42.csv'}),
 Document(page_content='31 декабря 2022,31 декабря 2021,,\n(6 mысячах poccuicкux рублей),Прим:,года,года\nАКТИВЫ,,,\nВнеоборотные активы,,,\nЗаймы выданные,29 386 665,41 030 353,\nОтложенный налоговый актив,13,4 082 948,2 067 136\nИтого внеоборотные активы,33 469 613,43 097 489,\nОборотные активы,,,\nЗаймы выданные,219 741,532 399,\nАвансы выданные,372,367,\nПрочая дебиторская задолженность,105,151,